<div hidden=True>
    author: Marco Angius
    company: TomorrowData srl
    mail: marco.angius@tomorrowdata.io
    notebook-version: nov20
    
</div>

# Hands-on 3: Supervised Learning
This section is meant for learning the Scikit-Learn APIs end provide a playground for machine learning supervised tasks.

[Scikit-Learn](https://scikit-learn.org/stable/index.html#) is a library for data mining and data analysis. It  includes models for classification, regression and clustering. It is built on top of NumPy. SciPy and matplotlib. 

For the purpose of this playground, to get familiar with the Scikit-Learn APis, we would use [Toy Datasets](https://scikit-learn.org/stable/datasets/index.html#toy-datasets) available in the library. 

Datasets in `sklearn.datasets` return a *Bunch*:
> Dictionary-like object, the interesting attributes are: ‘data’, the data to learn, ‘target’, the regression targets, ‘DESCR’, the full description of the dataset, and ‘filename’, the physical location of boston csv dataset (added in version 0.20).

### mglearn library 
For visualizing the results obtained with our models we are going to employ an existing library made by Andreas C. Muller (author of the book *Introduction to Machine Learning with Python*). The library is available in the [github repository](https://github.com/amueller/mglearn).

In [ ]:
from sklearn.datasets import load_boston, load_diabetes, load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import export_graphviz

from IPython.display import Image
from six import StringIO

import pydotplus
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
import math

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
import math

In [ ]:
R_STATE = 99

## Dataset load
We will use the **boston house prices** and the **diabetes** dataset for regressions tasks. The **iris** and the **breast cancer** are used for multi-class and binary-class classification tasks respectively. 

In [ ]:
boston_houses_ds = load_boston()
iris_ds = load_iris()
diabetes_ds = load_diabetes()
breast_ds = load_breast_cancer()

<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 1**
- check the datasets. Use `print(ds.DESCR)`to print information for each of them.

</div>

In [ ]:
print(boston_houses_ds.DESCR)

## K-Nearest Neighbor: Classification
The first model we are going to use is the KNN which is a very simple model to start with. KNN can be used for both for classification or regression tasks. Here we are going to see classification first. 

<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 2: Classification with KNN**
- split the breast cancer dataset in training and test sets 
- instantiate a KNN classifier specifying *n_neighbours* in the constructor
- fit the model on the training data
- make predictions on the test data
- observe the obtained performances
</div>

<div class="alert alert-success" role="alert">
    
<img src="./icons/lightbulb.png"  width="20" height="20" align="left"> &nbsp; **SciKit-Learn Tip 1: Models**

Models in scikit-learn follow all the same API. 

- to fit a model use the `model.fit(X_test, y_test)`
- to make predictions use the `model.predict(X_traing)`

In order to split the available data X,y in training and test use the `sklearn.model_selection.train_test_split(X, y)` function.

</div>

<div class="alert alert-success" role="alert">
    
<img src="./icons/lightbulb.png"  width="20" height="20" align="left"> &nbsp; **SciKit-Learn Tip 2: Metrics**

Also it is possible to evaluate the results by means of metrics. There are metrics for [classification](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics) and [regression](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics).

For now just use `metrics.accuracy_score(y_true, y_pred)` for classification tasks and `metrics.mean_squared_error(y_true, y_pred)` for regression tasks.

</div>


In [ ]:
print(breast_ds.DESCR)

In [ ]:
X, y = breast_ds.data, breast_ds.target
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=R_STATE)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3, weights="distance" )

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, knn.predict(X_test))

### KNN Model complexity

<div class="alert alert-warning" role="alert">
    
<img src="./icons/book.png"  width="20" height="20" align="left"> &nbsp;  **Theory: Model Complexity** 

The number of neighbors determines the complexity of the model. 

A lower value leads to a higher model complexity which means that the model would be very sensitive to noise (when a point of a class is in a region populated the most by the other class points). 

Vice versa, in the case of an higher number of neighbors used to evaluate the point's class the presence of noise is less relevant.
</div>

In [ ]:
def plot_visual_boundaries(dataset, ranges, model=KNeighborsClassifier):
    # use only the first 2 features if more than 2 features
    X, y = dataset.data[:, :2] if dataset.data.shape[1] > 2 else dataset.data, dataset.target
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=R_STATE)
    assert len(ranges) <= 3, "ranges lenght should be less or equal 3"
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    for n_neighbors, ax in zip(ranges, axes):
        cknn_ = model(n_neighbors=n_neighbors).fit(X_train, y_train)
        mglearn.plots.plot_2d_separator(cknn_, X_test, fill=True, eps=0.5, ax=ax, alpha=.4)
        mglearn.discrete_scatter(X_test[:, 0], X_test[:, 1], y_test, ax=ax)
        ax.set_title("{} neighbor(s)".format(n_neighbors), fontdict={'fontsize':15})
        ax.set_xlabel(dataset.feature_names[0], fontdict={'fontsize':13})
        ax.set_ylabel(dataset.feature_names[1], fontdict={'fontsize':13})

In [ ]:
plot_visual_boundaries(breast_ds, [1, 3, 30])

### Which k ?
It is possible to train different models on different values of k and observe the performance of the model on the test set. 

In [ ]:
def k_search_classification(dataset, ranges, random_state=R_STATE):
    X, y = dataset.data, dataset.target
    X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=random_state)
    
    training_accuracy = []
    test_accuracy = []
    neighbors_settings = ranges
    for n in neighbors_settings:
        cknn_ = KNeighborsClassifier(n_neighbors=n)
        cknn_.fit(X_train, y_train)
        training_accuracy.append(cknn_.score(X_train, y_train))
        test_accuracy.append(cknn_.score(X_test, y_test))
    
    plt.figure(figsize=[10, 5])
    plt.title("Accuracy for different k values", size=15)
    plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
    plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
    plt.ylabel("Accuracy", size=13)
    plt.xlabel("n_neighbors", size=13)
    plt.legend()

<hr>

## K-Nearest Neighbor: Regression


<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 3: Regression with KNN**
- split the diabetes dataset in training and test sets 
- instantiate a KNN classifier specifying *n_neighbours* in the constructor
- fit the model on the training data
- make predictions on the test data
- observe the obtained performances
</div>

### Regression Analysis 
We can observe how the model works on a single feature of the dataset on the given target.

In [ ]:
def plot_regression(dataset, ranges, f_number=5):
    X, y = dataset.data[::4, f_number], dataset.target[::4]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=R_STATE)
    assert len(ranges) <= 3, "ranges lenght should be less or equal 3"
    
    fig, axes = plt.subplots(1, 3, figsize=(20, 10))
    line = np.linspace(min(X_train), max(X_train), 1000).reshape(-1, 1)
    for n_neighbors, ax in zip(ranges, axes):
        rknn_ = KNeighborsRegressor(n_neighbors=n_neighbors)
        rknn_.fit(X_train.reshape(-1, 1), y_train)
        ax.plot(line, rknn_.predict(line))
        ax.plot(X_train, y_train, '^', c=mglearn.cm2(0), markersize=8)
        ax.plot(X_test, y_test, 'v', c=mglearn.cm2(1), markersize=8)
        ax.set_title("{} neighbor(s)\n train score: {:.2f} test score: {:.2f}".format(
            n_neighbors, 
            rknn_.score(X_train.reshape(-1,1), y_train),
            rknn_.score(X_test.reshape(-1,1), y_test)))
        ax.set_xlabel("Feature")
        ax.set_ylabel("Target")
        axes[0].legend(["Model predictions", "Training data/target","Test data/target"], loc="best")

<hr>

In [ ]:
def k_search_regression(dataset, ranges):
    X, y = dataset.data, dataset.target
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=R_STATE)
    
    training_accuracy = []
    test_accuracy = []
    neighbors_settings = ranges
    for n in neighbors_settings:
        rknn_ = KNeighborsRegressor(n_neighbors=n)
        rknn_.fit(X_train, y_train)
        training_accuracy.append(mean_squared_error(y_train, rknn_.predict(X_train)))
        test_accuracy.append(mean_squared_error(y_test, rknn_.predict(X_test)))
    
    plt.figure(figsize=[10, 5])
    plt.title("MSE for different k values", size=15)
    plt.plot(neighbors_settings, training_accuracy, label="training mse")
    plt.plot(neighbors_settings, test_accuracy, label="test mse")
    plt.ylabel("MSE", size=13)
    plt.xlabel("n_neighbors", size=13)
    plt.legend()

<hr>

## Linear Models
The next model we are going to use predict the target by means of a linear function of the input features.
The general formula for regression tasks is: 

$$ \hat{y} = w[0] * x[0] + w[1] * x[1] + \dots + w[p] * x[p] + b $$

We have also seen that in the case of classification tasks we can employ a linear model for defining decision boundaries: 

$$ \hat{y} = w[0] * x[0] + w[1] * x[1] + \dots + w[p] * x[p] + b > 0 $$

For now we focus on regression tasks only

<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 4: Linear Models**
- split the boston houses dataset in training and test sets 
- fit the model on the training data
- make predictions on the test data
- observe the obtained performances
- check the parameters' magnitude
</div>

In [ ]:
X, y = boston_houses_ds.data, boston_houses_ds.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=R_STATE)

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_train, y_train)

In [ ]:
lm.coef_

In [ ]:
r2_score(y_test, lm.predict(X_test))

<hr>

In [ ]:
def plot_coefficients_given_models(dataset, models):
    models = [models] if not isinstance(models, list) else models
    
    f = plt.figure(figsize=[12, 7])
    ax = f.add_subplot()
    for m in models:
        ax.plot(range(0, len(dataset.feature_names)), m.coef_, 's', 
                label=str(m.__class__).strip("<>''").split(".")[-1])
    
    ax.set_title("Coefficients magnitued", fontdict={'fontsize':15})
    ax.set_xlabel("Coefficient index", fontdict={'fontsize':13})
    ax.set_ylabel("Feature", fontdict={'fontsize':13})
    ax.hlines(0, 0, len(m.coef_))
    ax.set_xticks(range(0, len(dataset.feature_names)))
    ax.set_xticklabels(dataset.feature_names)
    xlabels = ax.get_xticklabels()
    ax.set_xticklabels(xlabels, rotation=90)
    plt.legend()

## Ridge e Lasso
Ordinary Least Square presents a regularization problem due to some of its coefficient magnitudes tends to assume high values fitting on the training data. Ridge and lasso add a constraint on the optimization function in order to limit these phenomena. 

The Ridge regularization: 
$$ \min_{w} || X w - y||_2^2 + \alpha ||w||_2^2 $$

The Lasso Regularization: 
$$ \min_{w} { \frac{1}{2n_{\text{samples}}} ||X w - y||_2 ^ 2 + \alpha ||w||_1} $$

<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 5: Linear Models with regularization**  

Try to repeat the regression experiment using `Ridge` and compare the performances with the `LinearRegression` model.

To see the different coefficient magnitudes use the `plot_coefficients_given_models` with a given dataset and pass a list of trained models in order to compare the results.
</div>

In [ ]:
print(boston_houses_ds.DESCR)

## Decision Tree
The last model we are going to see in this notebook is the Decision Tree for classification. We can compare the different scores obtained with the different classifiers and see witch performs better.

Please note that the decision tree has more hyperparameters in comparison with the K-Nearest Neighbor. The ones you can play with are: 
- `max_depth`: controls the complexity of the model
- `min_sample_split`: min number of samples to allow a split in an internal node. If the node is not split it is considered as a leaf, thus the distribution of samples in the leaf would affect the final decision.


<div class="alert alert-info" role="alert">
    
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp;  **Exercise 4: Decision Trees**
- split the breast dataset in training and test sets 
- fit the model on the training data
- make predictions on the test data
- observe the obtained performances
</div>

### Visualize Decision Tree Structure
It is possible to check how the tree has been built. Use the below `plot_decision_tree(dataset, model)` function to plot the tree. You can also try with different parameters and see how those impact the tree construction.

<div class="alert alert-success" role="alert">
    
<img src="./icons/lightbulb.png"  width="20" height="20" align="left"> &nbsp; **SciKit-Learn Tip 3: Reproducibility**

Sometimes we want to have the same output across different runs of the algorithm. Due to the majority of them uses pseudo random generators it is possible to fix the seed of those using the `random_state` parameter.

</div>

In [ ]:
def plot_decision_tree(dataset, model):
    dot_data = StringIO()
    export_graphviz(model, out_file=dot_data,
                    feature_names=dataset.feature_names,
                    class_names=True,
                    filled=True, rounded=True,
                    special_characters=True)

    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    return Image(graph.create_png())

In [ ]:
iris_ds.target_names

In [ ]:
plot_decision_tree(iris_ds, dt)

## Homeworks

<div class="alert alert-danger" role="alert">
    
<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp;  **Task 1.1**:  Using the `KNeighborsClassifier` model try to repeat the training using the **iris** dataset.
- compute the score using the `accuracy_score`

[**SOLUTION**](./solutions/handson3/solution_1.1.py)
</div>

<div class="alert alert-danger" role="alert">
    
<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp;  **Task 1.2**:  Continure the analysis by manipulating some of the features.
- add new features by combining existing ones (see the iris dataset description for some insights).
- remove existing features.

To help you manipulating the dataset, I have provided you with a function which gives you a `pandas.DataFrame` X_df with the features and a `pandas.Series` with the target.

[**SOLUTION**](./solutions/handson3/solution_1.2.py)
</div>

In [ ]:
def create_dataframe_series(dataset): 
    return (
        pd.DataFrame(dataset.data, columns=[x.replace(" ", "_") for x in dataset.feature_names]), 
        pd.Series(dataset.target, name="target")
    )

<div class="alert alert-danger" role="alert">
    
<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp;  **Task 2**:   Using the `KNeighborsRegressor` model try to repeat the training using the **Boston Houses** dataset.
- compute the score using the `r2_score` or the `mean_squared_error`

[**SOLUTION**](./solutions/handson3/solution_2.py)

</div>

<div class="alert alert-danger" role="alert">
    
<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp;  **Task 3**:   Using the `DecisionTreeRegressor` model try to repeat the training using the **Boston Houses** dataset.
- compute the score using the `r2_score` or the `mean_squared_error`
- try different parametrization for the Tree in order to observe the performances. [API](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

[**SOLUTION**](./solutions/handson3/solution_3.py)
</div>

<div class="alert alert-danger" role="alert">
    
<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp;  **Task 4**: Using the **Breast Cancer**  dataset: 

- try to repeat the regression experiment using the `Lasso` model and compare the performances with the `LinearRegression` and `Ridge` models.
- use `plot_coefficients_given_models` with the Breast Cancer dataset and pass a list of trained models in order to compare the results.
[**SOLUTION**](./solutions/handson3/solution_4.py)
</div>

<div hidden=True>
<img src="./icons/list.png"  width="20" height="20" align="left"> &nbsp; Icon made by <a href="https://www.flaticon.com/authors/smashicons" title="Smashicons">Smashicons</a> from <a href="https://www.flaticon.com/"             title="Flaticon">www.flaticon.com</a>

<img src="./icons/lightbulb.png"  width="20" height="20" align="left"> &nbsp;Icon made by <a href="https://www.flaticon.com/authors/pixelmeetup" title="Pixelmeetup">Pixelmeetup</a> from <a href="https://www.flaticon.com/"             title="Flaticon">www.flaticon.com</a>

<img src="./icons/new.png"  width="20" height="20" align="left"> &nbsp; Icon made by <a href="https://www.flaticon.com/authors/pixel-perfect" title="Pixel perfect">Pixel perfect</a> from <a href="https://www.flaticon.com/"             title="Flaticon">www.flaticon.com</a>

<img src="./icons/chemistry.png"  width="20" height="20" align="left"> &nbsp; Icon made by <a href="https://www.flaticon.com/authors/popcorns-arts" title="Icon Pond">Icon Pond</a> from <a href="https://www.flaticon.com/"             title="Flaticon">www.flaticon.com</a>

<img src="./icons/book.png"  width="20" height="20" align="left"> &nbsp; Icon made by <a href="https://www.flaticon.com/authors/popcorns-arts" title="Icon Pond">Icon Pond</a> from <a href="https://www.flaticon.com/"             title="Flaticon">www.flaticon.com</a>
    
</div>